# Composable Indices Demo

In [1]:
from gpt_index import GPTTreeIndex, GPTSimpleKeywordTableIndex, GPTListIndex, SimpleDirectoryReader

/Users/egafni/Library/Caches/pypoetry/virtualenvs/nlp-G_cUgREO-py3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Load Datasets

Load both the NYC Wikipedia page as well as Paul Graham's "What I Worked On" essay

In [9]:
# fetch "New York City" page from Wikipedia
from pathlib import Path

import requests
response = requests.get(
    'https://en.wikipedia.org/w/api.php',
    params={
        'action': 'query',
        'format': 'json',
        'titles': 'New York City',
        'prop': 'extracts',
        # 'exintro': True,
        'explaintext': True,
    }
).json()
page = next(iter(response['query']['pages'].values()))
nyc_text = page['extract']

# data_path = Path('data')
# if not data_path.exists():
#     Path.mkdir(data_path)
!mkdir -p data/test_wiki/
with open('data/test_wiki/nyc_text.txt', 'w') as fp:
    fp.write(nyc_text)

In [11]:
# load NYC dataset
nyc_documents = SimpleDirectoryReader('data/test_wiki/').load_data()

In [12]:
# load PG's essay
essay_documents = SimpleDirectoryReader('../paul_graham_essay/data/').load_data()

FileNotFoundError: [Errno 2] No such file or directory: '../paul_graham_essay/data'

### Building the document indices
Build a tree index for the NYC wiki page and PG essay

In [ ]:
# build NYC index
nyc_index = GPTTreeIndex(nyc_documents)

In [5]:
nyc_index.save_to_disk('../test_wiki/index.json')

In [ ]:
# build essay index
essay_index = GPTTreeIndex(essay_documents)

In [7]:
essay_index.save_to_disk('../paul_graham_essay/index.json')

### Loading the indices
Build a tree indices for the NYC wiki page and PG essay

In [2]:
# try loading
nyc_index = GPTTreeIndex.load_from_disk('../test_wiki/index.json')
essay_index = GPTTreeIndex.load_from_disk('../paul_graham_essay/index.json')

### Set summaries for the indices

Add text summaries to indices, so we can compose other indices on top of it

In [3]:
nyc_index.set_text("""
    New York, often called New York City or NYC, 
    is the most populous city in the United States. 
    With a 2020 population of 8,804,190 distributed over 300.46 square miles (778.2 km2), 
    New York City is also the most densely populated major city in the United States, 
    and is more than twice as populous as second-place Los Angeles. 
    New York City lies at the southern tip of New York State, and 
    constitutes the geographical and demographic center of both the 
    Northeast megalopolis and the New York metropolitan area, the 
    largest metropolitan area in the world by urban landmass.[8] With over 
    20.1 million people in its metropolitan statistical area and 23.5 million 
    in its combined statistical area as of 2020, New York is one of the world's 
    most populous megacities, and over 58 million people live within 250 mi (400 km) of 
    the city. New York City is a global cultural, financial, and media center with 
    a significant influence on commerce, health care and life sciences, entertainment, 
    research, technology, education, politics, tourism, dining, art, fashion, and sports. 
    Home to the headquarters of the United Nations, 
    New York is an important center for international diplomacy,
    an established safe haven for global investors, and is sometimes described as the capital of the world.
""") 
essay_index.set_text("""
    Author: Paul Graham. 
    The author grew up painting and writing essays. 
    He wrote a book on Lisp and did freelance Lisp hacking work to support himself. 
    He also became the de facto studio assistant for Idelle Weber, an early photorealist painter. 
    He eventually had the idea to start a company to put art galleries online, but the idea was unsuccessful. 
    He then had the idea to write software to build online stores, which became the basis for his successful company, Viaweb. 
    After Viaweb was acquired by Yahoo!, the author returned to painting and started writing essays online. 
    He wrote a book of essays, Hackers & Painters, and worked on spam filters. 
    He also bought a building in Cambridge to use as an office. 
    He then had the idea to start Y Combinator, an investment firm that would 
    make a larger number of smaller investments and help founders remain as CEO. 
    He and his partner Jessica Livingston ran Y Combinator and funded a batch of startups twice a year. 
    He also continued to write essays, cook for groups of friends, and explore the concept of invented vs discovered in software. 

""")

### Build Keyword Table Index on top of tree indices! 

We set summaries for each of the NYC and essay indices, and then compose a keyword index on top of it.

In [4]:
# set query config
query_configs = [
    {
        "index_struct_type": "tree",
        "query_mode": "default",
        "query_kwargs": {
            "child_branch_factor": 2
        }
    },
    {
        "index_struct_type": "keyword_table",
        "query_mode": "simple",
        "query_kwargs": {}
    },
]

In [ ]:
keyword_table = GPTSimpleKeywordTableIndex([nyc_index, essay_index], max_keywords_per_chunk=50)

In [ ]:
# ask it a question about NYC 
response = keyword_table.query(
    "What is the climate of New York City like? How cold is it during the winter?", 
    mode="recursive", 
    query_configs=query_configs
)

In [12]:
response

'The climate of New York City is temperate. It is cold during the winter, with an average temperature of 16 degrees Celsius. Spring and autumn are unpredictable and can range from cool to warm, although they are usually mild with low humidity. Summers are typically hot and humid, with a daily mean temperature of 77.5 degrees Celsius in July. Nighttime temperatures are often enhanced due to the urban heat island effect. Daytime temperatures exceed 90 degrees Fahrenheit on average of 17 days each summer and in some years exceed 100 degrees Fahrenheit, although this is a rare achievement. Similarly, readings of 0 degrees Fahrenheit are also extremely rare. Extreme temperatures have ranged from -15 degrees Fahrenheit to 106 degrees Fahrenheit. The average water temperature of the nearby Atlantic Ocean ranges from 39.7 degrees Fahrenheit in February to 74.1 degrees Fahrenheit in August.'

In [ ]:
# ask it a question about PG's essay
response = keyword_table.query(
    "What did the author do growing up, before his time at Y Combinator?", 
    mode="recursive", 
    query_configs=query_configs
)

In [10]:
response

'The author was a writer and programmer who studied art and worked on building a WYSIWYG site builder.'